In [21]:
import json
import requests
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any
from dotenv import load_dotenv
import os
from datetime import timezone

API_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/config/api_keys.env"
KEY_NAME = "CRYPTO_PANIC_API_KEY"

BASE_URL = "https://cryptopanic.com/api/v1/posts/"

MAX_PER_ASSET = 5
DAYS_BACK = 7

def load_symbols_from_token_list(path: str) -> List[str]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    tokens = data.get("tokens", [])
    symbols = list({token["symbol"] for token in tokens if "symbol" in token})
    return symbols

def load_api_key(api_path: str, key_name: str) -> str:
    """Load API key from environment file"""
    load_dotenv(api_path)
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Missing {key_name} in environment")
    return api_key

def save_json(data: object, path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def get_recent_articles(symbol: str, filter_type: str) -> List[Dict[str, Any]]:

    api_key = load_api_key(API_PATH, KEY_NAME)

    params = {
        "auth_token": api_key,
        "currencies": symbol,
        "filter": filter_type
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"[{symbol}] Error {response.status_code} with filter={filter_type}")
        return []

    cutoff = datetime.now(timezone.utc) - timedelta(days=DAYS_BACK)
    results = []
    for post in response.json().get("results", []):
        try:
            pub_time = datetime.strptime(post["published_at"], "%Y-%m-%dT%H:%M:%S%z")
        except Exception:
            continue
        if pub_time >= cutoff:
            results.append({
                "text": post["title"],
                "source": post["url"],
                "published_at": pub_time.isoformat()
            })
        if len(results) >= MAX_PER_ASSET:
            break
    return results

def fetch_news_for_symbols(symbols: List[str]) -> List[Dict[str, Any]]:
    output_data = []
    cutoff = (datetime.utcnow() - timedelta(days=DAYS_BACK)).isoformat()

    for symbol in symbols:
        articles = get_recent_articles(symbol, filter_type="hot")
        if not articles:
            print(f"[{symbol}] Fallback to filter=new")
            articles = get_recent_articles(symbol, filter_type="new")

        if articles:
            output_data.append({
                "asset_name": symbol,
                "status": "analyzed",
                "texts": articles
            })
        else:
            output_data.append({
                "asset_name": symbol,
                "status": "no_recent_data",
                "texts": [],
                "risk_level": None,
                "reason": f"No qualifying articles found after {cutoff}",
                "priority": None
            })

        time.sleep(1)

    return output_data

In [22]:
ASSET_FILE_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/input/assets_input.json"
OUTPUT_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/output/cp_output.json"

if __name__ == "__main__":
    symbols = load_symbols_from_token_list(ASSET_FILE_PATH)
    print(f"🔍 Detected symbols: {symbols}")
    asset_data = fetch_news_for_symbols(symbols)
    save_json(asset_data, OUTPUT_PATH)
    print(f"✅ Output saved to {OUTPUT_PATH}")

🔍 Detected symbols: ['ETH', 'USDT', 'USDC', 'PEPE', 'GNO']


/var/folders/6y/01p567bn48j6qsm68vtwjy2w0000gn/T/ipykernel_3657/4275959100.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cutoff = (datetime.utcnow() - timedelta(days=DAYS_BACK)).isoformat()


[USDC] Fallback to filter=new
[PEPE] Fallback to filter=new
[GNO] Fallback to filter=new
✅ Output saved to /Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/output/cp_output.json
